<a href="https://colab.research.google.com/github/RaquelRufino/InformationRetrieval/blob/master/ri_lab_06/lab06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
result = pd.read_csv("https://raw.githubusercontent.com/RaquelRufino/InformationRetrieval/master/ri_lab_04/results.csv")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

###Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; 

In [0]:
M = result.text.count()
for word in indexes:
  k = len(indexes[word])
  IDF = round(np.log((M+1)/k),2)
  indexes[word].append(IDF)

###Implementar as seguintes versões do modelo vetorial:(10 pts)

    Representação binária;
    

In [0]:

def binary_vsm(query, document):
  score = 0
  query_tokens = query.split()
  doc_tokens = document.split()
  
  for token in query_tokens:
    score += (token in doc_tokens)
    
  return score

TF (lembre-se que esse modelo já está implementado);


In [0]:
def tf_vsm(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    score += doc_tokens.count(word)
  
  return score

    TF-IDF;


In [0]:
def tfidf_vsm(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    score += cwd * indexes[word][-1]
  
  return round(score,2)

BM25* (não usaremos Okapi já que os documentos não tem grande variação de tamanho).

In [0]:
def bm25_vsm(query, document, k):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = len(indexes[word][:-1])
    score += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw))
  
  return round(score,2)

### Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;

In [0]:
consultas = ['educação', 'governo', 'bolsonaro']

In [0]:
def top5Documentos(consulta):
  n = 0
  d_binaria = []
  d_tf = []
  d_tfidf = []
  d_bm25 = []
  for documento in result.text:
    documento = documento.lower()
    n += 1
    bisect.insort(d_binaria, binary_vsm(consulta, documento), n)
    bisect.insort(d_tf, (tf_vsm(consulta, documento), n))
    bisect.insort(d_tfidf, (tfidf_vsm(consulta, documento), n))
    bisect.insort(d_bm25, (bm25_vsm(consulta, documento, 10), n))
  
  d_binaria.reverse()
  d_tf.reverse()
  d_tfidf.reverse()
  d_bm25.reverse()
  
  return d_binaria[:5], d_tf[:5], d_tfidf[:5], d_bm25[:5]

In [0]:

top5_binary = ['','','']
top5_tf = ['','','']
top5_tfidf = ['','','']
top5_bm25 = ['','','']

top5_binary[0], top5_tf[0], top5_tfidf[0], top5_bm25[0] = create_top5_models(consultas[0])
top5_binary[1], top5_tf[1], top5_tfidf[1], top5_bm25[1] = create_top5_models(consultas[1])
top5_binary[2], top5_tf[2], top5_tfidf[2], top5_bm25[2] = create_top5_models(consultas[2])

In [85]:
query_df = pd.DataFrame()

query_df['Query'] = consultas
query_df['Binary'] = top5_binary
query_df['TF'] = top5_tf
query_df['TF-IDF'] = top5_tfidf
query_df['BM25'] = top5_bm25

query_df.index+=1
query_df

,Query,Binary,TF,TF-IDF,BM25
1,educação,"[(1, 248), (1, 240), (1, 239), (1, 233), (1, 2...","[(15, 221), (8, 222), (6, 239), (6, 130), (5, ...","[(32.85, 221), (17.52, 222), (13.14, 239), (13...","[(6.28, 221), (4.65, 222), (3.92, 239), (3.92,..."
2,governo,"[(1, 249), (1, 248), (1, 246), (1, 245), (1, 2...","[(14, 173), (12, 166), (10, 248), (9, 115), (8...","[(12.46, 173), (10.68, 166), (8.9, 248), (8.01...","[(2.47, 173), (2.31, 166), (2.12, 248), (2.01,..."
3,bolsonaro,"[(1, 248), (1, 240), (1, 238), (1, 237), (1, 2...","[(32, 151), (30, 207), (30, 166), (19, 19), (1...","[(42.24, 151), (39.6, 207), (39.6, 166), (25.0...","[(4.79, 151), (4.72, 207), (4.72, 166), (4.12,..."


### Compare os resultados encontrados e responda.

#### Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta

In [0]:

def get_top1_score(top5):
  return [top[0][0] for top in top5]

def get_top1_doc(top5):
  return [top[0][1] for top in top5]



In [0]:
score_top5_binary = get_top1_score(top5_binary)
score_top5_tf = get_top1_score(top5_tf)
score_top5_tfidf = get_top1_score(top5_tfidf)
score_top5_bm25 = get_top1_score(top5_bm25)

doc_top5_binary = get_top1_doc(top5_binary)
doc_top5_tf = get_top1_doc(top5_tf)
doc_top5_tfidf = get_top1_doc(top5_tfidf)
doc_top5_bm25 = get_top1_doc(top5_bm25)



**Representação binária**

In [90]:
binary_res = pd.DataFrame()
binary_res['Query'] = queries
binary_res['Document'] = doc_top5_binary
binary_res['Binary Score'] = score_top5_binary

binary_res.index += 1
binary_res

,Query,Document,Binary Score
1,Sergio Moro,248,1
2,Presidente Bolsonaro,249,1
3,Paulo Guedes,248,1


**TF**

In [91]:
tf_res = pd.DataFrame()
tf_res['Query'] = queries
tf_res['Document'] = doc_top5_tf
tf_res['TF Score'] = score_top5_tf

tf_res.index += 1
tf_res

,Query,Document,TF Score
1,Sergio Moro,221,15
2,Presidente Bolsonaro,173,14
3,Paulo Guedes,151,32


**TF-IDF**

In [93]:
tfidf_res = pd.DataFrame()
tfidf_res['Query'] = queries
tfidf_res['Document'] = doc_top5_tfidf
tfidf_res['TFIDF Score'] = score_top5_tfidf

tfidf_res.index += 1
tfidf_res

,Query,Document,TFIDF Score
1,Sergio Moro,221,32.85
2,Presidente Bolsonaro,173,12.46
3,Paulo Guedes,151,42.24


**BM-25**

In [95]:

bm25_res = pd.DataFrame()
bm25_res['Query'] = queries
bm25_res['Document'] = doc_top5_bm25
bm25_res['BM25 Score'] = score_top5_bm25

bm25_res.index += 1
bm25_res

,Query,Document,BM25 Score
1,Sergio Moro,221,6.28
2,Presidente Bolsonaro,173,2.47
3,Paulo Guedes,151,4.79


O TD-IDF teve o melhor resultado para as 3 consultas consideradas.


#### Calcule e reporte o overlap par-a-par entre os resultados de cada modelo.

In [0]:

def inter_size(a,b):
  return len([elem for elem in a if elem in b])

def jaccard_index(a,b):
  n_a = len(a)
  n_b = len(b)
  n_ab = inter_size(a,b)
  exp = n_a + n_b - n_ab
  jaccard = n_ab / exp if exp != 0 else 0
  return jaccard

In [97]:
all_docs = []
all_docs.append(top5_binary)
all_docs.append(top5_bm25)
all_docs.append(top5_tf)
all_docs.append(top5_tfidf)

matrix = [['Measure', 'Binary', 'TF', 'TF-IDF', 'BM25']]
titles = matrix
for i in range(len(all_docs)):
  line = []
  for j in range(len(all_docs)):
    jaccard = jaccard_index(all_docs[i], all_docs[j])
    line.append(round(jaccard,2))
  matrix.append(line)
for i in range(1, len(matrix)):
  matrix[i].insert(0, matrix[0][i])
  
pd.DataFrame(matrix)

,0,1,2,3,4
0,Measure,Binary,TF,TF-IDF,BM25
1,Binary,1,0,0,0
2,TF,0,1,0,0
3,TF-IDF,0,0,1,0
4,BM25,0,0,0,1


O indice de Jaccard mostrou que os resultados de todos os modelos foram diferentes, o que pode ser atribuído à sua ingenuidade.